# Подготовка ноутбука 

## Пробрасываем magic methods

In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

## Imports

In [2]:
import pandas as pd 
import numpy as np 
from data.prepare_data import PrepareData
from dotenv import load_dotenv
import os 
from pathlib import Path
from warnings import filterwarnings 
import torch 
import torchvision
from torch.utils.data import DataLoader
from IPython.display import display
import pytesseract
import shutil
try:
    from PIL import Image
except ImportError:
     import Image
import cv2
import requests
import json
import os
from datetime import datetime
import csv


import pandas as pd 
import numpy as np 
from data.prepare_data import PrepareData
from dotenv import load_dotenv
import os 
from pathlib import Path
from warnings import filterwarnings 
import torch 
import torchvision
from torch.utils.data import DataLoader
from IPython.display import display
import pytesseract
import shutil
try:
    from PIL import Image
except ImportError:
     import Image
import cv2
import requests
import json
import os
from datetime import datetime
import csv
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock
import hashlib
from utils.s3_optimize import S3Manager
import logging

from geo.download_images import MapillaryS3Client

## Нужные переменные 

In [3]:
# Настройка логирования
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
ROOT_DIR = Path('../../')
load_dotenv()
filterwarnings(action='ignore')

# Загрузка датафрейма

In [ ]:
def main():
    """Пример использования клиента с разбивкой bbox"""
    load_dotenv()
    
    # Инициализация S3 менеджера
    s3_manager = S3Manager(
        max_workers=10,
        chunk_size=32 * 1024 * 1024
    )
    
    # Инициализация Mapillary клиента
    mapillary_client = MapillaryS3Client(
        access_token=os.getenv('API_MAPILLARY_KEY'),
        s3_manager=s3_manager,
        max_workers=10,
        cache_dir=ROOT_DIR / 'logs/download_data/cache_moscow_images'
    )
    
    # Основной bbox для Москвы
    moscow_oblast_bbox = [36.866323, 55.334271, 38.564470, 56.191264]
    moscow_bbox = [37.366723, 55.579170, 37.841304, 55.907936]
    
    # Вариант 1: Автоматическое разбиение большого bbox
    logger.info("=== Вариант 1: Автоматическое разбиение большого bbox ===")
    images = mapillary_client.get_images_for_large_area(
        bbox=moscow_bbox,
        max_results_per_bbox=1500,
        grid_size=20,
        use_cache=True
    )
    
    # Вариант 2: Сетка вокруг центральной точки
    # logger.info("=== Вариант 2: Сетка вокруг центральной точки ===")
    # moscow_center = (55.7558, 37.6173)  # Кремль
    # images = mapillary_client.get_images_around_point(
    #     center_lat=moscow_center[0],
    #     center_lon=moscow_center[1],
    #     grid_radius=3,  # 7x7 = 49 bbox
    #     bbox_size=0.01,  # Размер каждого bbox
    #     max_results_per_bbox=300
    # )
    
    if images:
        logger.info(f"Найдено {len(images)} изображений. Начинаем загрузку в S3...")
        
        # Загружаем в S3
        start_time = time.time()
        results = mapillary_client.download_and_upload_to_s3(
            images=images,
            s3_prefix="site/raw_data",
            image_format="JPEG", 
            quality=80
        )
        
        duration = time.time() - start_time
        logger.info(f"Загрузка завершена за {duration:.2f} секунд")
        
        # Сохраняем метаданные
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        mapillary_client.save_metadata_to_csv(
            images, 
            ROOT_DIR / 'data/processed_data/moscow_images.csv',
        )


if __name__ == "__main__":
    main()

INFO:__main__:=== Вариант 1: Автоматическое разбиение большого bbox ===
INFO:geo.download_images:Основной bbox разбит на 400 частей
INFO:geo.download_images:  Часть 1: [37.366723, 55.57917, 37.39045205, 55.595608299999995]
INFO:geo.download_images:  Часть 2: [37.366723, 55.595608299999995, 37.39045205, 55.6120466]
INFO:geo.download_images:  Часть 3: [37.366723, 55.6120466, 37.39045205, 55.6284849]
INFO:geo.download_images:  Часть 4: [37.366723, 55.6284849, 37.39045205, 55.6449232]
INFO:geo.download_images:  Часть 5: [37.366723, 55.6449232, 37.39045205, 55.6613615]
INFO:geo.download_images:  Часть 6: [37.366723, 55.6613615, 37.39045205, 55.677799799999995]
INFO:geo.download_images:  Часть 7: [37.366723, 55.677799799999995, 37.39045205, 55.6942381]
INFO:geo.download_images:  Часть 8: [37.366723, 55.6942381, 37.39045205, 55.7106764]
INFO:geo.download_images:  Часть 9: [37.366723, 55.7106764, 37.39045205, 55.7271147]
INFO:geo.download_images:  Часть 10: [37.366723, 55.7271147, 37.39045205,

INFO:geo.download_images:  Часть 16: [37.366723, 55.8257445, 37.39045205, 55.842182799999996]
INFO:geo.download_images:  Часть 17: [37.366723, 55.842182799999996, 37.39045205, 55.8586211]
INFO:geo.download_images:  Часть 18: [37.366723, 55.8586211, 37.39045205, 55.8750594]
INFO:geo.download_images:  Часть 19: [37.366723, 55.8750594, 37.39045205, 55.8914977]
INFO:geo.download_images:  Часть 20: [37.366723, 55.8914977, 37.39045205, 55.907936]
INFO:geo.download_images:  Часть 21: [37.39045205, 55.57917, 37.4141811, 55.595608299999995]
INFO:geo.download_images:  Часть 22: [37.39045205, 55.595608299999995, 37.4141811, 55.6120466]
INFO:geo.download_images:  Часть 23: [37.39045205, 55.6120466, 37.4141811, 55.6284849]
INFO:geo.download_images:  Часть 24: [37.39045205, 55.6284849, 37.4141811, 55.6449232]
INFO:geo.download_images:  Часть 25: [37.39045205, 55.6449232, 37.4141811, 55.6613615]
INFO:geo.download_images:  Часть 26: [37.39045205, 55.6613615, 37.4141811, 55.677799799999995]
INFO:geo.do

In [ ]:
s3_manager = S3Manager(
        max_workers=4,
        chunk_size=16 * 1024 * 1024
    )

s3_files = s3_manager.list_files(prefix="site/raw_data/", file_extensions=[".jpg", ".jpeg", ".png"])
download_results = s3_manager.batch_download_files(
    s3_files,
    local_dir=ROOT_DIR / 'data/processed_data/moscow_image',
    progress_callback=lambda current, total: print(f"Download progress: {current}/{total}")
)